In [47]:
%load_ext autoreload
%autoreload 2
from pcfg_parse_gen import Pcfg, PcfgGenerator, CkyParse
import nltk
import benepar
from nltk.tree import Tree
from nltk.corpus.reader import BracketParseCorpusReader
import re
import pandas as pd

In [48]:
benepar.download('benepar_en')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package benepar_en to
[nltk_data]     /Users/junbo/nltk_data...
[nltk_data]   Package benepar_en is already up-to-date!
[nltk_data] Downloading package punkt to /Users/junbo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/junbo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Generate tags and assign weights for all allowed words

**Steps:**<br>
>**1. Generate parsed trees using "benepar" parser based on devset and other sentences we found**<br>
>**2. For each word in allowed words, find tags in the parsed trees and count the occurances of the tags**<br>
>**3. Manually assign the tags to those words that appeared in allowed words but not in the parsed trees**<br>
>**4. Fix the weights of each word**<br>
>**5. Manually fix the format and the tags for punctuations (e.g. replace left hand side "." to END)**

Combine devset and the new sentences we found into a list

In [52]:
allQuotes = []
file3 = open('devset.txt', 'r')
file4 = open('quotes_new_preprocessed.txt', 'r')
for i in file3:
    allQuotes.append(i.strip().split(' '))
for j in file4:
    allQuotes.append(list(filter(None, j.strip().split(' '))))
file3.close()
file4.close()

Generate parsed trees using "benepar" parser, and write to allTrees.txt

In [ ]:
bst_parser = benepar.Parser("benepar_en")
count = 0
file5 = open('allTrees.txt', 'w')
for sent in allQuotes:
    file5.write(str(bst_parser.parse(sent)))
    file5.write("\n")
    count+=1
    print(count)
    print(sent)
file5.close()

For each word in allowed words, find and keep all associate tags in the parsed trees. <br>
If a word cannot be found in the parsed trees, then assign tag Misc and weight 1 to that word. <br>
If only one tag found for a word, then assign weight 1 to it. <br>
If multiple tags found for a word, then count the occurance of each tag, and assign the count as the weight. <br>

In [34]:
file6 = open('allTrees.txt', 'r')
file7 = open('allowed_words.txt', 'r')
file8 = open('tags.txt', 'w')
allTrees = file6.read()
all_tags = set()
for word in file7:
    tag_line = '1    '
    wrd1 = word.strip()
    tags = list()
    if (wrd1 != '(') and (wrd1 != ')'):
        for i in re.findall("\([^\s]+ "+str(wrd1), allTrees):
            tg = i.split(" ")[0].replace("(",'')
            if tg != '':
                tags.append(tg)
    else:
        if wrd1 == '(':
            tag_line+=("(")
        else:
            tag_line+=(")")
        tag_line+='    '
        
        tag_line+= wrd1
        file8.write(tag_line)
        file8.write("\n") 
        continue
    
    if (len(tags)==0) and (wrd1 != '(') and (wrd1 != ')'):
        tag_line+=("Misc")
        tag_line+='    '
        tag_line+= wrd1
        file8.write(tag_line)
        file8.write("\n")
    else:
        all_tags = all_tags.union(set(tags))
        for j in list(set(tags)):
            tag_line = str(tags.count(j))
            if len(set(tags))==1:
                tag_line = str(1)
            tag_line+= '    '
            tag_line+=(j)
            tag_line+='    '
            tag_line+= wrd1
            file8.write(tag_line)
            file8.write("\n")
    
file6.close()
file7.close()
file8.close()

For each word, convert the weight of each tag to percentage

In [62]:
tagsDF = pd.read_csv('tags.txt', header=None, sep='    ', engine='python', names=['ct','tag','word'])\
.sort_values(by=['tag']).reset_index(drop=True)

merged = tagsDF.groupby('word').sum().rename(columns={'ct':'summed'}).reset_index().merge(tagsDF)

merged['weight'] = (merged['ct']/merged['summed']*100).astype(int)

weights = []
for i in merged['weight']:
    if i==100:
        weights.append(i)
    else:
        weights.append(i+1)

merged['weight'] = weights

merged[['weight', 'tag', 'word']].to_csv('tags_weight.gr', header=False, index=False, sep='|')